In [2]:
# Imports

# Matemaricos, mapiulacao de dados e funcoes
import numpy as np
import pandas as pd
import time
# Desligando alertas de chained_assignment
pd.options.mode.chained_assignment = None  # default='warn'

# Graficos
#import matplotlib
#import matplotlib.pyplot as plt
#import seaborn as sns

# Chamadas e tratamentos de dados
import re
import requests

In [3]:
# Carregamento da listagem de propostas
listagem = pd.DataFrame(pd.read_excel('../Listagem/listagem_12.xlsx'))

In [4]:
# Limpeza de colunas vazias
listagem = listagem.drop(listagem.columns[[1,3,5,7,9]], axis=1)

# Antigo
#listagem = listagem.drop('Unnamed: 1', axis=1)
#listagem = listagem.drop('Unnamed: 3', axis=1)
#listagem = listagem.drop('Unnamed: 5', axis=1)
#listagem = listagem.drop('Unnamed: 7', axis=1)
#listagem = listagem.drop('Unnamed: 9', axis=1)

In [5]:
# Limpeza de linhas varias (NaN) - A SER FEITO
listagem = listagem.dropna()

In [6]:
#Criacao das colunas de codigo do tipo e id do projeto
listagem['Cod_Tipo'] = ''
listagem['Id_Projeto'] = ''


In [7]:
# Preenchimento do campo Cod_Tipo
for i in listagem.index:
    #print(listagem["Tipo"][i])
    if(listagem["Tipo"][i] == "Requerimento"):
        listagem['Cod_Tipo'][i] = 'REQ'
    elif(listagem["Tipo"][i] == "Projeto de Lei"):
        listagem['Cod_Tipo'][i] = 'PLEI'
    elif(listagem["Tipo"][i] == "Projeto de Lei Complementar"):
        listagem['Cod_Tipo'][i] = 'PLEIC'
    elif(listagem["Tipo"][i] == "Representação"):
        listagem['Cod_Tipo'][i] = 'PLEI'
    elif(listagem["Tipo"][i] == "Projeto de Resolução"):
        listagem['Cod_Tipo'][i] = 'PRES'
    elif(listagem["Tipo"][i] == "Pedido de Informação"):
        listagem['Cod_Tipo'][i] = 'PEDIF'
    elif(listagem["Tipo"][i] == "Moção"):
        listagem['Cod_Tipo'][i] = 'MOC'
    elif(listagem["Tipo"][i] == "Mensagem do Executivo (Projeto de Lei)"):
        listagem['Cod_Tipo'][i] = 'MSGPL'
    elif(listagem["Tipo"][i] == "Indicação"):
        listagem['Cod_Tipo'][i] = 'IND'
    elif(listagem["Tipo"][i] == "Projeto de Decreto Legislativo"):
        listagem['Cod_Tipo'][i] = 'PDL'
    elif(listagem["Tipo"][i] == "Mensagem do Executivo"):
        listagem['Cod_Tipo'][i] = 'MSGEX'
    elif(listagem["Tipo"][i] == "Mensagem do Executivo (Projeto de Lei Complementar)"):
        listagem['Cod_Tipo'][i] = 'MSGPC'
    elif(listagem["Tipo"][i] == "Prestação de Contas"):
        listagem['Cod_Tipo'][i] = 'PTCTS'
    elif(listagem["Tipo"][i] == "Projeto de Emenda à Lei Orgânica"):
        listagem['Cod_Tipo'][i] = 'PELOR'
    elif(listagem["Tipo"][i] == "Projeto de Lei de Iniciativa Popular"):
        listagem['Cod_Tipo'][i] = 'PLEIP'
    elif(listagem["Tipo"][i] == "Projeto de Resolução - Comissão de Exame Denúncia"):
        listagem['Cod_Tipo'][i] = 'PRCED'
    elif(listagem["Tipo"][i] == "Proposta de Revisão da Lei Orgânica Municipal"):
        listagem['Cod_Tipo'][i] = 'PRLOM'
    elif(listagem["Tipo"][i] == "Proposta de Revisão do Regimento Interno"):
        listagem['Cod_Tipo'][i] = 'PRRI'
    elif(listagem["Tipo"][i] == "Requerimento Popular"):
        listagem['Cod_Tipo'][i] = 'RPO'
    elif(listagem["Tipo"][i] == "Requerimento de CPI"):
        listagem['Cod_Tipo'][i] = 'RCPI'
    else:
        listagem['Cod_Tipo'][i] = 'DESCONHECIDO'

In [8]:

# Variaveis de requisicao
num_proj = int(listagem["Projeto"][2])
ano_proj = int(listagem["Ano"][2])
tipo_proj = listagem["Cod_Tipo"][2]

# Requisicao para o portal de propostas da Camara
query = {"numero": num_proj, "ano": ano_proj, "tipo": tipo_proj}
#query = {"numero": 193600, "ano": 2012, "tipo": 'REQ'}
resposta = requests.get('https://www.camarajf.mg.gov.br/sal/listapropos.php?', params=query)

# Limpeza para pegar o id do projeto dentro da prefeitura
retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))
id_projeto = re.search('PROPOSICAO.PHP(.+?)TARGET=BLANK', retorno).group(1).replace('?NUM=','').replace('"','')

listagem['Id_Projeto'][2] = id_projeto

In [47]:
# Consulta via rest - TESTES 
# Preenchimento do campo Cod_Tipo

for i in listagem.index:
    
    if listagem['Id_Projeto'][i] == "": 
    
        # Variaveis de requisicao
        num_proj = int(listagem["Projeto"][i])
        ano_proj = int(listagem["Ano"][i])
        tipo_proj = listagem["Cod_Tipo"][i]

        # Requisicao para o portal de propostas da Camara
        query = {"numero": num_proj, "ano": ano_proj, "tipo": tipo_proj}
        #query = {"numero": 193600, "ano": 2012, "tipo": 'REQ'}
        resposta = requests.get('https://www.camarajf.mg.gov.br/sal/listapropos.php?', params=query)

        # Limpeza para pegar o id do projeto dentro da prefeitura
        retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))
        id_projeto = re.search('PROPOSICAO.PHP(.+?)TARGET=BLANK', retorno).group(1).replace('?NUM=','').replace('"','')

        listagem['Id_Projeto'][i] = id_projeto


In [49]:

count = 0
for i in listagem.index:
    
    if listagem['Id_Projeto'][i] == "":
        print(listagem['Projeto'][i], listagem['Cod_Tipo'][i], listagem['Ano'][i], listagem['Id_Projeto'][i])
        print(count)
        break
    else:
        count = count + 1

In [61]:
i = 6500

print(listagem['Projeto'][i], listagem['Cod_Tipo'][i], listagem['Ano'][i], listagem['Id_Projeto'][i])

1.0 PELOR 2012.0 37776


In [48]:
display(listagem)

,Projeto,Ano,Tipo,Autor,Ementa,Situação,Cod_Tipo,Id_Projeto
0,204700.0,2012.0,Requerimento,Chico Evangelista,Solicita a construção de um muro na rua Alexan...,Em Tramitação,REQ,10975
2,193600.0,2012.0,Requerimento,Noraldino,Solicita a capina e limpeza na Av. Barão do Ri...,Em Tramitação,REQ,34734
4,185800.0,2012.0,Requerimento,Betão,Solicita posteamento e energia elétrica na rua...,Em Tramitação,REQ,27822
6,176700.0,2012.0,Requerimento,Chico Evangelista,Solicita a colocação de quebra-molas na rua Pa...,Em Tramitação,REQ,31332
8,56300.0,2012.0,Requerimento,Fiorilo,Solicita a substituição das lampadas vapor de ...,Em Tramitação,REQ,30452
...,...,...,...,...,...,...,...,...
6500,1.0,2012.0,Projeto de Emenda à Lei Orgânica,Noraldino Júnior,Altera a redação do art. 2º do Ato das Disposi...,Arquivada,PELOR,37776
6502,1.0,2012.0,Moção,Ana do Padre Frederico,Moção de Aplauso - Mariano Augusto Alonso de A...,Arquivada,MOC,37776
6506,1.0,2012.0,Requerimento,Chico Evangelista,Solicita que asfalte a Rua Expedicionário Anto...,Em Tramitação,REQ,37776
6508,1.0,2012.0,Representação,Noraldino,Solicita apoio na construção do Posto Avançado...,Aprovado,PLEI,37776
